In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
try:
    df = pd.read_csv('../data/complete_user_data.csv', index_col=0)
except FileNotFoundError as fnfError:
    print(fnfError)

In [58]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2)

In [59]:
df.head()

,spotted,battles_on_stunning_vehicles,hits,battle_avg_xp,draws,max_xp,survived_battles,wins,losses,capture_points,...,max_frags,shots,frags,max_damage,xp,avg_damage_assisted,piercings,user_id,nickname,wn8
0,807,38,3564,263,15,1116,174,398,396,522,...,5,7319,389,1567,212897,108.00,2126,630675552,greenbirdus,"567,79"
1,1530,3,9382,234,26,1239,611,1014,1029,3847,...,6,19333,1233,2012,483650,111.85,6362,596561562,C_ABTOBA3A,"530,66"
2,128,0,1500,147,2,726,130,265,254,446,...,5,4138,157,1036,76622,42.06,883,634554120,Babay19910403,"117,49"
3,14488,105,56828,382,118,2160,1985,5282,5098,8549,...,10,96897,7643,6255,4005621,297.38,35147,502086999,2803nevs,"1134,25"
4,498,0,4965,408,3,2181,202,321,197,348,...,10,8304,895,2232,212310,80.20,3878,534287717,ce3peo,"2172,30"


In [60]:
def dropIrrelevantCols(df, cols):
    return df.drop(columns=cols)
df_train = dropIrrelevantCols(df_train, ['user_id', 'nickname'])
df_test = dropIrrelevantCols(df_test, ['user_id', 'nickname'])

In [61]:
def convertColsToFloat(df, cols):
    for column in cols:
        try:
            df[column] = df[column].astype(float)
        except:
            df[column] = df[column].str.replace(',', '.')
            df[column] = df[column].astype(float)
            
    return df
df_train = convertColsToFloat(df_train, list(df_train.columns))
df_test = convertColsToFloat(df_test, list(df_test.columns))

In [62]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

X_train, y_train = df_train.drop(columns=['wn8']), df_train['wn8']
X_test, y_test = df_test.drop(columns=['wn8']), df_test['wn8']
numericalAttribs = X_train.select_dtypes(np.number)
numericalAttribsNames = list(numericalAttribs.columns)
numericalPipeline = make_pipeline(SimpleImputer(strategy='median'),
                                  StandardScaler())

preprocessing = make_column_transformer((numericalPipeline, numericalAttribsNames))

In [63]:
numericalAttribsNames

['spotted',
 'battles_on_stunning_vehicles',
 'hits',
 'battle_avg_xp',
 'draws',
 'max_xp',
 'survived_battles',
 'wins',
 'losses',
 'capture_points',
 'battles',
 'damage_dealt',
 'damage_received',
 'max_frags',
 'shots',
 'frags',
 'max_damage',
 'xp',
 'avg_damage_assisted',
 'piercings']

In [64]:
from sklearn.metrics import mean_absolute_error, max_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor

def trainAndScore(models, X_train, X_test, y_train, y_test):
    modelsScore = {}
    for name, model in models.items():
        metricsScore = {}
        model.fit(X_train, y_train)
        
        meanAbsErr = abs(cross_val_score(model, X_test, y_test, scoring='neg_mean_absolute_error', cv=3)).mean()
        maxErr = abs(cross_val_score(model, X_test, y_test, scoring='max_error', cv=3)).mean()
        metricsScore.update({'Mean Absolute Error':meanAbsErr})
        metricsScore.update({'Max Error':maxErr})
        
        modelsScore.update({name:metricsScore})
    
    return modelsScore

def applyPreprocessing(model, preprocessing):
    modelWithPreprocessing = make_pipeline(preprocessing,
                                           model)
    return modelWithPreprocessing

candidateModels = {'Lasso':applyPreprocessing(Lasso(), preprocessing),
                   'Elastic Net':applyPreprocessing(ElasticNet(), preprocessing),
                   'Random Forest':applyPreprocessing(RandomForestRegressor(), preprocessing)}

scores = trainAndScore(candidateModels, X_train, X_test, y_train, y_test)

/home/bruno/tools/miniconda3/envs/py-data-science/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.907e+05, tolerance: 3.138e+04
  model = cd_fast.enet_coordinate_descent(
/home/bruno/tools/miniconda3/envs/py-data-science/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.893e+05, tolerance: 5.314e+03
  model = cd_fast.enet_coordinate_descent(
/home/bruno/tools/miniconda3/envs/py-data-science/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterati

In [65]:
scores

{'Lasso': {'Mean Absolute Error': 136.67257895739382,
  'Max Error': 765.9582871817171},
 'Elastic Net': {'Mean Absolute Error': 172.18412265040243,
  'Max Error': 824.3741104112396},
 'Random Forest': {'Mean Absolute Error': 164.64960404255316,
  'Max Error': 762.6205333333334}}